In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from kaggle_secrets import UserSecretsClient
import os
user_secrets=UserSecretsClient()
os.environ["KAGGLE_USERNAME"]=user_secrets.get_secret("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"]=user_secrets.get_secret("KAGGLE_KEY")

In [2]:
import os
os.environ["KERAS_BACKEND"]="jax" #This sets the backend for Keras to "jax."By default,Keras can use TensorFlow as its backend,but specifying "jax" switches the backend to JAX.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" #This sets the fraction of the GPU memory that should be allocated for the XLA (Accelerated Linear Algebra) Python client. By setting it to "1.00," you're specifying that 100% of the GPU memory to be used.

In [3]:
#Importing libraries
import keras
import keras_hub
import keras_nlp

In [4]:
#Combines the use of both FP32 and lower bit floating points(such as FP16) to reduce memory footprint during model training,resulting in improved performance
keras.mixed_precision.set_global_policy('mixed_bfloat16')

In [5]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2025-02-25 16:57:05--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.80, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1740506225&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDUwNjIyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIwO

In [6]:
import json
data=[]
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features=json.loads(line)
        if features["context"]:
            continue
        template="Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))
data=data[:1000] #Taking only the first 1000 lines in the dataset

In [7]:
gemma_lm=keras_hub.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [8]:
#Inference before fine-tuning using TopKSampler
prompt=template.format(
    instruction="Write a short story about a robot discovering human emotions.",
    response="",
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Write a short story about a robot discovering human emotions.

Response:
My story is about a robot that was built to do one thing. But one day, it was discovered by a scientist that the robot could feel emotions like happiness, sadness, and anger.

The robot was very excited about this discovery. It wanted to know more about these emotions. But the scientist was afraid that this would cause the robot to become self-aware. The scientist decided to put the robot in a room with different emotions. The robot was very excited about this. It wanted to explore and learn about all the different emotions.

But as it explored, it became more and more confused. It realized that these emotions were very different from each other. It also realized that they were very complex. It started to feel overwhelmed.

The scientist decided to take the robot out of the room and put it in a different room. The robot felt very sad. It wanted to know why it was so sad. The scientist explained to the

In [9]:
#Inference before fine tuning using TopPSampler
prompt=template.format(
    instruction="Imagine a future where artificial intelligence surpasses human intelligence.How would this affect global politics,ethics,and daily life? Provide both positive and negative outcomes.",
    response=""
)
sampler=keras_nlp.samplers.TopPSampler(p=0.9,seed=10) 
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Imagine a future where artificial intelligence surpasses human intelligence.How would this affect global politics,ethics,and daily life? Provide both positive and negative outcomes.

Response:
Answer: In my opinion, when human intelligence reaches artificial intelligence, life will become more interesting. We can use AI in many ways in our daily lives such as to detect lies, assist in decision-making and also for entertainment purposes. Positive effects of having artificial intelligence are making our life more enjoyable, easier, and saving time to do other activities. AI is like having a personal assistant who can help us do anything such as preparing a shopping list, helping to find the correct location or giving us directions.  Negative effects can be seen as we lose our unique personality, the rise of job loss and social isolation. With this new technology we may become more dependent on technology to do tasks, that can also become addictive. So the main point of view 

In [10]:
#Inference before fine tuning using BeamSampler
prompt=template.format(
    instruction="Summarize the benefits of renewable energy in three bullet points.",
    response=""
)
sampler=keras_nlp.samplers.BeamSampler(num_beams=3)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Summarize the benefits of renewable energy in three bullet points.

Response:
The benefits of renewable energy are numerous and far-reaching. Renewable energy sources, such as solar, wind, and hydroelectric power, are clean, sustainable, and abundant. They do not emit greenhouse gases or contribute to climate change, making them an important tool in the fight against global warming. Renewable energy sources are also cost-effective, as they do not require the use of fossil fuels, which are expensive and finite resources. Additionally, renewable energy sources can provide a reliable and stable source of energy, reducing the need for backup power sources and increasing energy security. Finally, renewable energy sources can create jobs and stimulate economic growth, as they require the development of new technologies and infrastructure. Overall, the benefits of renewable energy are numerous and far-reaching, making it an important tool in the fight against climate change and t

In [11]:
#Fine tuning the model with LoRa
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [12]:
#Limit the input sequence length to 256 to control memory usage
gemma_lm.preprocessor.sequence_length=256
#Use AdamW optimizer a commom optimizer for tuning transformer models
optimizer=keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
#Exclude layer norm and bias term from decay
optimizer.exclude_from_weight_decay(var_names=["bias","scale"])
#Model compilation with specific loss functions and weighted metrics
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [13]:
#Fit the model for tuning 
gemma_lm.fit(data,epochs=5,batch_size=1)

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 482s 452ms/step - loss: 0.8413 - sparse_categorical_accuracy: 0.5375
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 452s 430ms/step - loss: 0.7155 - sparse_categorical_accuracy: 0.5726
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 430s 430ms/step - loss: 0.6958 - sparse_categorical_accuracy: 0.5806
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 430s 430ms/step - loss: 0.6751 - sparse_categorical_accuracy: 0.5895
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 430s 430ms/step - loss: 0.6494 - sparse_categorical_accuracy: 0.6011


In [14]:
#Inference after fine tuning....we will be asking the same questions after tuning and see the results
prompt=template.format(
    instruction="Write a short story about a robot discovering human emotions.",
    response="",
)
sampler=keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Write a short story about a robot discovering human emotions.

Response:
One day, a robot named R2-D2 was exploring the galaxy, looking for new planets to discover. He came across a small planet called Tatooine, which was inhabited by human beings. R2-D2 was fascinated by the human emotions he saw, and he was curious to learn more about them.
R2-D2 approached a human settlement on Tatooine and introduced himself, asking if he could learn more about their culture and way of life. The humans were curious about R2-D2's origins, but they welcomed him with open arms and offered to help him understand more about human emotions.
Over the next few days, R2-D2 observed and listened to the various emotions that humans experienced, such as joy, sadness, anger, and fear. He was amazed at how complex and nuanced these emotions could be, and how they could affect someone's life.
One day, R2-D2 was helping a group of humans who were celebrating a special event. He noticed that they were 

In [15]:
#Inference after fine tuning using TopPSampler
prompt=template.format(
    instruction="Imagine a future where artificial intelligence surpasses human intelligence.How would this affect global politics,ethics,and daily life? Provide both positive and negative outcomes",
    response=""
)
sampler=keras_nlp.samplers.TopPSampler(p=0.9,seed=10) 
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Imagine a future where artificial intelligence surpasses human intelligence.How would this affect global politics,ethics,and daily life? Provide both positive and negative outcomes

Response:
First,the development of AI will open up unprecedented possibilities for improvement of human lives.With computers and other machines able to perform tasks for us,there would be greater time and resources for us to do more important things,such as providing better healthcare,solving more problems,and improving ourselves.Furthermore,the bar for physical and mental strength would be lowered,making it much easier for more people to achieve greater heights. 
On the other hand,a rise of artificial intelligence could mean less jobs for humans.Though they could be redeployed to complete more work for humans, there might still be fewer jobs overall, leading to increased unemployment and stagnated economies.Additionally, AI could pose a threat to national security as foreign adversaries gain a

In [16]:
#Inference after fine tuning using BeamSampler
prompt=template.format(
    instruction="Summarize the benefits of renewable energy in three bullet points.",
    response=""
)
sampler=keras_nlp.samplers.BeamSampler(num_beams=3)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction:
Summarize the benefits of renewable energy in three bullet points.

Response:
1. Renewable energy is a clean source of energy.
2. Renewable energy is a sustainable source of energy.
3. Renewable energy is a cost-effective source of energy.


In [ ]:
#To get better responses from the fine-tuned model,you can experiment with:

#1)Increasing the size of the fine-tuning dataset
#2)Training for more steps(epochs)
#3)Setting a higher LoRA rank
#4)Modifying the hyperparameter values such as learning_rate and weight_decay.